# Задание 2

## Построение и оценка бейзлайна

### Разбиение данных

In [100]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import os

In [113]:
# заного загружаем набор данных
csv_file = next((f for f in os.listdir() if f.endswith(".csv")), None)
if not csv_file:
    print("файл не найден.")

df = pd.read_csv(csv_file)

подготовим данные.

у нас используется много категориальных переменных, поэтому не будем возиться с labelenc, а воспользуемся типичным pandas.get_dummies

In [114]:
# подготовим данные
df_final = pd.concat(
    [
        df.select_dtypes(include=["number"]),
        df[
            [
                col
                for col in df.select_dtypes(include=["object"])
                if df[col].nunique() < 5
            ]
        ],
    ],
    axis=1,
)

categorical_cols = df_final.select_dtypes(include=["object"]).columns
df_encoded = pd.get_dummies(df_final, columns=categorical_cols, drop_first=True)

# выберем таргет
X = df_encoded.drop(columns="Churn_Yes")
y = df_encoded["Churn_Yes"]

X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(
    X, y, test_size=0.3, random_state=52, stratify=y
)
print(f"разбиение train/test: {X_train.shape[0]} | {X_test.shape[0]}")

разбиение train/test: 4930 | 2113


### Константная модель(и)

In [115]:
from sklearn.dummy import DummyClassifier
from sklearn import metrics as sk_metrics

dummy_models = [
    DummyClassifier(strategy=S)
    for S in DummyClassifier._parameter_constraints["strategy"][0].options
    if S != "constant"
]

for m in dummy_models:
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)

    acc = sk_metrics.accuracy_score(y_test, y_pred)
    f1 = sk_metrics.f1_score(y_test, y_pred)
    roc = sk_metrics.roc_auc_score(y_test, y_pred)

    print(f"модель : {m.strategy:^12}\t| acc={acc:.3f}; f1={f1:.3f}; roc={roc:.3f};")

модель :    prior    	| acc=0.735; f1=0.000; roc=0.500;
модель : most_frequent	| acc=0.735; f1=0.000; roc=0.500;
модель :   uniform   	| acc=0.502; f1=0.348; roc=0.501;
модель :  stratified 	| acc=0.632; f1=0.315; roc=0.532;


по метрикам - все плохо. константные модели работают чуть лучше подбрасывания монетки.

### Baseline

возьмем несколько моделей, обучим и проверим их метрики

In [111]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

models = {
    "Logistic Regression": LogisticRegression(random_state=52, max_iter=2000),
    "Decision Tree": DecisionTreeClassifier(random_state=52),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(random_state=52, max_depth=10, n_jobs=16),
    "Gradient Boosting": GradientBoostingClassifier(
        random_state=52, learning_rate=0.001, n_estimators=1000
    ),
}

for n, m in models.items():
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)

    acc = sk_metrics.accuracy_score(y_test, y_pred)
    f1 = sk_metrics.f1_score(y_test, y_pred)
    roc = sk_metrics.roc_auc_score(y_test, y_pred)

    print(f"модель : {n:^20}\t| acc={acc:.3f}; f1={f1:.3f}; roc={roc:.3f};")

модель : Logistic Regression 	| acc=0.807; f1=0.593; roc=0.718;
модель :    Decision Tree    	| acc=0.726; f1=0.482; roc=0.648;
модель : K-Nearest Neighbors 	| acc=0.782; f1=0.561; roc=0.700;
модель :    Random Forest    	| acc=0.805; f1=0.573; roc=0.706;
модель :  Gradient Boosting  	| acc=0.777; f1=0.364; roc=0.606;


уже что-то, с чем можно работать. за базовое решение возьмем модель Random Forest